### 暫時看新聞用 
過濾掉 EPS < 1 和 YoY < 10% 的標的 

In [4]:
from finlab import data
from finlab.dataframe import FinlabDataFrame
import pandas as pd
from datetime import datetime
import os

# 參數設定
length = 24
std_dev = 1
required_follow_up = 5
must_fulfill_full_count = False  # True 時強制要有滿足的後續根數才能通過
eps_level = 1
yoy_level = 5 # 10 means 10%

data.force_cloud_download = False
data.truncate_start = '2024-09-01'
# data.truncate_end = '2025-05-13'

long_result = []

def save_daily_list_pd(data, common_int_sorted, filename="daily_list.csv"):
    today = data.index[-1].strftime('%Y-%m-%d')
    weekday = datetime.now().strftime("%A")   # Monday, Tuesday...

    # 新的一筆資料
    df_new = pd.DataFrame({
        "date": [today],
        "weekday": [weekday],
        "values": [common_int_sorted]
    })
    
    # 若 CSV 不存在 → 直接建立
    if not os.path.exists(filename):
        df_new.to_csv(filename, index=False)
        return
    
    # 若 CSV 存在 → 先讀出
    df = pd.read_csv(filename)

    # 將 values 從字串轉回 list（避免重複 encoding）
    try:
        df["values"] = df["values"].apply(lambda x: ast.literal_eval(x))
    except:
        pass

    # 刪除今天舊資料
    df = df[df["date"] != today]
    
    # 加入今天新的
    df = pd.concat([df, df_new], ignore_index=True)
    
    # 儲存
    df.to_csv(filename, index=False)

# 限定只抓取上市櫃普通股 (排除ETF、ETN、特別股、存託憑證) 的股價。
with data.universe(market='TSE_OTC'):
    open_prices_all = data.get('price:開盤價')
    high_prices_all = data.get('price:最高價')
    low_prices_all = data.get('price:最低價')
    close_prices_all = data.get('price:收盤價')
    eps = data.get('financial_statement:每股盈餘') # 取得 EPS DataFrame
    rev_year_growth = data.get('monthly_revenue:去年同月增減(%)')
last_eps = eps.ffill().iloc[-1]  # 先 forward fill，然後取最後一列（最近的有效日期）
last_yoy = rev_year_growth.ffill().iloc[-1]

symbols_list = list(open_prices_all.columns)

for index, symbol in enumerate(symbols_list):

    # # 跳過不處理 symbol 不在 last_eps 裡面的資料
    # if symbol not in last_eps:
    #     print(f'第{index}個標的 - {symbol} ⚠️ 無 EPS 資料，跳過')
    #     continue

    # # 跳過 EPS < 0 的標的
    # if last_eps[symbol] < eps_level:
    #     print(f'第{index}個標的 - {symbol} ⚠️ EPS < 1，跳過')
    #     continue

    # 跳過不處理 symbol 不在 last_yoy 裡面的資料
    if symbol not in last_yoy:
        print(f'第{index}個標的 - {symbol} ⚠️ 無 YoY 資料，跳過')
        continue

    if last_yoy[symbol] < yoy_level:
        print(f'第{index}個標的 - {symbol} ⚠️ YoY < 10%，跳過')
        continue
        
    print(f'第{index}個標的 - {symbol}')
    
    open_prices = open_prices_all[[symbol]].tail(28)
    high_prices = high_prices_all[[symbol]].tail(28)
    low_prices = low_prices_all[[symbol]].tail(28)
    close_prices = close_prices_all[[symbol]].tail(28)

    #把開高低收合併成一個 Dataframe
    data = pd.DataFrame({
        'open': open_prices[symbol],
        'high': high_prices[symbol],
        'low': low_prices[symbol],
        'close': close_prices[symbol],
        'YoY': last_yoy[symbol],
        # 'eps': last_eps[symbol]
    }, index=open_prices.index )
    
    if data.shape[0] < 4:
        print(f"{symbol} 的資料筆數不足 4 筆，跳過該標的分析")
        continue

    data['MA'] = data['close'].rolling(window=length).mean()  # 移動平均線
    data['STD'] = data['close'].rolling(window=length).std(ddof=0)

    data['Upper_Band'] = data['MA'] + std_dev * data['STD']   # 上軌
    data['Lower_Band'] = data['MA'] - std_dev * data['STD']   # 下軌

    # 檢查是否符合突破條件 + 後續K棒站穩條件
    for x in range(-5, 0):  # 檢查最近5根
        try:
            prev_open = data.loc[data.index[x - 1], 'open']
            prev_close = data.loc[data.index[x - 1], 'close']
            prev_middle = (prev_open + prev_close) / 2
            prev_upper = data.loc[data.index[x - 1], 'Upper_Band']

            curr_open = data.loc[data.index[x], 'open']
            curr_close = data.loc[data.index[x], 'close']
            curr_middle = (curr_open + curr_close) / 2
            curr_upper = data.loc[data.index[x], 'Upper_Band']
        except:
            continue

        # 條件 1：前一根在上軌下方
        if prev_open < prev_upper and prev_middle < prev_upper:

            # 條件 2：當根突破布林上軌
            if (curr_open < curr_close and curr_middle >= curr_upper) or \
               (curr_open > curr_upper and curr_close > curr_upper):

                # 檢查後續K棒條件
                remaining_kbars = abs(x) - 1
                if must_fulfill_full_count and remaining_kbars < required_follow_up:
                    print(f'{symbol} - 不足 {required_follow_up} 根後續K棒，略過（強制驗證開啟）')
                    continue

                actual_check_count = min(remaining_kbars, required_follow_up)

                if actual_check_count == 0:
                    print(f'✅ {symbol} - 突破點為最後一根，無後續K棒可驗證，直接加入 ✅')
                    long_result.append(symbol)
                    break

                follow_through_ok = True
                for i in range(1, actual_check_count + 1):
                    next_open = data.loc[data.index[x + i], 'open']
                    next_close = data.loc[data.index[x + i], 'close']
                    next_upper = data.loc[data.index[x + i], 'Upper_Band']

                    if next_open <= next_upper or next_close <= next_upper:
                        follow_through_ok = False
                        print(f"❌ {symbol} - 第{x + i}根不符合，open 或 close 未高於上軌 ❌")
                        break

                if follow_through_ok:
                    print(f'✅ {symbol} - 符合條件（突破 + 後續 {actual_check_count} 根站上），加入 long_result ✅')
                    long_result.append(symbol)
                    break

# 把結果拿去比對股票期貨的標的，有重複的在產出。
stockinfo = pd.read_excel("2_stockinfo.ods", engine="odf")
stockinfo_list = stockinfo['Unnamed: 2'].astype(str).tolist()

common = list(set(long_result) & set(stockinfo_list))

# 轉成 int 並排序
common_int_sorted = sorted(map(int, common))
long_result = sorted(map(int, set(long_result)))
save_daily_list_pd(data, common_int_sorted)

print('-----------------------------------')
print(data)
print()
print(f"一般 List: {long_result}\n")
print(f"股期 List: {common_int_sorted}")


第0個標的 - 1101 ⚠️ YoY < 10%，跳過
第1個標的 - 1102 ⚠️ YoY < 10%，跳過
第2個標的 - 1103 ⚠️ YoY < 10%，跳過
第3個標的 - 1104 ⚠️ YoY < 10%，跳過
第4個標的 - 1107
第5個標的 - 1108 ⚠️ YoY < 10%，跳過
第6個標的 - 1109 ⚠️ YoY < 10%，跳過
第7個標的 - 1110 ⚠️ YoY < 10%，跳過
第8個標的 - 1201 ⚠️ YoY < 10%，跳過
第9個標的 - 1203 ⚠️ YoY < 10%，跳過
第10個標的 - 1210 ⚠️ YoY < 10%，跳過
第11個標的 - 1213
第12個標的 - 1215
第13個標的 - 1216 ⚠️ YoY < 10%，跳過
第14個標的 - 1217 ⚠️ YoY < 10%，跳過
第15個標的 - 1218
第16個標的 - 1219
第17個標的 - 1220 ⚠️ YoY < 10%，跳過
第18個標的 - 1225 ⚠️ YoY < 10%，跳過
第19個標的 - 1227 ⚠️ YoY < 10%，跳過
第20個標的 - 1229
第21個標的 - 1231 ⚠️ YoY < 10%，跳過
第22個標的 - 1232 ⚠️ YoY < 10%，跳過
第23個標的 - 1233 ⚠️ YoY < 10%，跳過
第24個標的 - 1234 ⚠️ YoY < 10%，跳過
第25個標的 - 1235 ⚠️ YoY < 10%，跳過
第26個標的 - 1236 ⚠️ YoY < 10%，跳過
第27個標的 - 1240 ⚠️ YoY < 10%，跳過
第28個標的 - 1256
第29個標的 - 1258
第30個標的 - 1259 ⚠️ YoY < 10%，跳過
第31個標的 - 1262
第32個標的 - 1264 ⚠️ YoY < 10%，跳過
第33個標的 - 1268 ⚠️ YoY < 10%，跳過
第34個標的 - 1294
第35個標的 - 1295
第36個標的 - 1301 ⚠️ YoY < 10%，跳過
第37個標的 - 1303 ⚠️ YoY < 10%，跳過
第38個標的 - 1304 ⚠️ YoY < 10%，跳過
第39個標的 - 1305 ⚠️